# Language model
AIFFEL 대전 탐색 노드 6    
LM, 즉 다음 단어를 예측하는 모델이다.

프로세스는 다음과 같다.     
1. Import package
2. Data loading  
3. Data preprocessing 
4. Data division
5. Modeling

## 1. Import package

In [1]:
from sklearn.model_selection import train_test_split
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os
import glob

## 2. Data loading

In [2]:
# Step 2. 데이터 읽어오기
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['\ufeffbaby It was all a dream', 'I used to read Word Up magazine', 'Salt n Pepa and Heavy D up in the limousine']


## 3. Data preprocessing
먼저, 특수문자를 제거하고 문장의 시작과 끝을 알려주는 태그를 붙여준다.   
그 후 토크나이저를 활용하여 12,000개의 단어만 사용을 하고 문장을 숫자로 변환한다.   
그리고 문장 단어가 15개 이상일 경우에는 제외한다.

In [3]:
# Step 3. 데이터 정제
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
print(len(corpus))
corpus[:10]

175749


['<start> baby it was all a dream <end>',
 '<start> i used to read word up magazine <end>',
 '<start> salt n pepa and heavy d up in the limousine <end>',
 '<start> hangin pictures on my wall <end>',
 '<start> every saturday rap attack mr magic marley marl <end>',
 '<start> i let my tape rock til my tape popped <end>',
 '<start> smokin weed and bambu sippin on private stock <end>',
 '<start> way back when i had the red and black lumberjack <end>',
 '<start> with the hat to match <end>',
 '<start> remember rappin duke duhha duhha <end>']

In [4]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    len15_tensor = []
    for txt in tensor:
        if len(txt) < 16:
            len15_tensor.append(txt)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(len15_tensor, maxlen=15, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer
#print(tensor.shape)
tensor, tokenizer = tokenize(corpus)

[[   2   51   11 ...    0    0    0]
 [   2    5  291 ...    0    0    0]
 [   2 3196  489 ...    0    0    0]
 ...
 [   2  135   50 ...    0    0    0]
 [   2  135    4 ...    0    0    0]
 [   2  135    4 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f00990b2890>


In [5]:
# check the tokenizer
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [6]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2  51  11  57  24   9 339   3   0   0   0   0   0   0]
[ 51  11  57  24   9 339   3   0   0   0   0   0   0   0]


## 4. Data division
훈련과 평가를 위해 데이터를 분리한다.

In [7]:
# Step 4. 평가 데이터셋 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input,test_size=0.2,shuffle=True)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Train:", enc_val.shape)
print("Target Train:", dec_val.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)
Source Train: (31203, 14)
Target Train: (31203, 14)


In [8]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
print(train_dataset)
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
print(val_dataset)

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>
<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>


## 5. Modeling
batch normalization과 rnn layer의 수를 하이퍼 파라미터로 설정하여 여러 모델로 실험을 하였다.

In [10]:
# Step 5. 인공지능 만들기
class TextGenerator(tf.keras.Model):
    # bn : batchnormalization (TRUE, FALSE), siz : rnn layer size (1~2)
    def __init__(self, vocab_size, embedding_size, hidden_size, bn, siz):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        if bn:
            self.bn_1 = tf.keras.layers.BatchNormalization()
        if siz > 1:
            self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
            if bn:
                self.bn_2 = tf.keras.layers.BatchNormalization()
        self.linear = tf.keras.layers.Dense(vocab_size)
        
        self.v_siz = vocab_size
        #self.emb_siz = embedding_size
        self.h_siz = hidden_size
        
        self.bn = bn
        self.siz = siz
        
    def call(self, x):
        out = self.embedding(x)
        #print(out)
        if self.siz > 0:
            out =  self.rnn_1(out)
            if self.bn:
                out = self.bn_1(out)
        
        if self.siz > 1:
            out = self.rnn_2(out)
            if self.bn:
                out = self.bn_2(out)
        
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model_01 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size, 0, 1) # rnn 1 no bn
model_02 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size, 0, 2) # rnn 2 no bn
model_11 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size, 1, 1) # rnn 1 + bn
model_12 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size, 1, 2) # rnn 2 + bn

In [11]:
for src_sample, tgt_sample in train_dataset.take(1): break
model_01(src_sample)
model_01.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 20,620,257
Trainable params: 20,620,257
Non-trainable params: 0
_________________________________________________________________


In [11]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,reduction='none')

In [12]:
model_01.compile(loss=loss, optimizer=optimizer)
history_01 = model_01.fit(train_dataset, epochs=10,validation_data=val_dataset,
                    verbose=1)

Epoch 1/10
487/487 [==============================] - 47s 96ms/step - loss: 3.4919 - val_loss: 3.0761
Epoch 2/10
487/487 [==============================] - 47s 96ms/step - loss: 2.9652 - val_loss: 2.8848
Epoch 3/10
487/487 [==============================] - 47s 96ms/step - loss: 2.7865 - val_loss: 2.7619
Epoch 4/10
487/487 [==============================] - 47s 97ms/step - loss: 2.6345 - val_loss: 2.6644
Epoch 5/10
487/487 [==============================] - 48s 99ms/step - loss: 2.4908 - val_loss: 2.5867
Epoch 6/10
487/487 [==============================] - 50s 102ms/step - loss: 2.3526 - val_loss: 2.5169
Epoch 7/10
487/487 [==============================] - 50s 102ms/step - loss: 2.2197 - val_loss: 2.4628
Epoch 8/10
487/487 [==============================] - 50s 102ms/step - loss: 2.0954 - val_loss: 2.4189
Epoch 9/10
487/487 [==============================] - 49s 101ms/step - loss: 1.9805 - val_loss: 2.3827
Epoch 10/10
487/487 [==============================] - 50s 102ms/step - loss: 

In [13]:
for src_sample, tgt_sample in train_dataset.take(1): break
model_11(src_sample)
model_11.summary()

Model: "text_generator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  5246976   
_________________________________________________________________
batch_normalization (BatchNo multiple                  4096      
_________________________________________________________________
dense_2 (Dense)              multiple                  12301025  
Total params: 20,624,353
Trainable params: 20,622,305
Non-trainable params: 2,048
_________________________________________________________________


In [14]:
model_11.compile(loss=loss, optimizer=optimizer)
history_11 = model_11.fit(train_dataset, epochs=10,validation_data=val_dataset,
                    verbose=1)

Epoch 1/10
487/487 [==============================] - 51s 104ms/step - loss: 3.3378 - val_loss: 2.9302
Epoch 2/10
487/487 [==============================] - 50s 103ms/step - loss: 2.7587 - val_loss: 2.7163
Epoch 3/10
487/487 [==============================] - 50s 103ms/step - loss: 2.5487 - val_loss: 2.6098
Epoch 4/10
487/487 [==============================] - 50s 103ms/step - loss: 2.3842 - val_loss: 2.5392
Epoch 5/10
487/487 [==============================] - 50s 103ms/step - loss: 2.2455 - val_loss: 2.4968
Epoch 6/10
487/487 [==============================] - 48s 99ms/step - loss: 2.1254 - val_loss: 2.4639
Epoch 7/10
487/487 [==============================] - 48s 99ms/step - loss: 2.0202 - val_loss: 2.4516
Epoch 8/10
487/487 [==============================] - 48s 98ms/step - loss: 1.9289 - val_loss: 2.4481
Epoch 9/10
487/487 [==============================] - 48s 99ms/step - loss: 1.8491 - val_loss: 2.4434
Epoch 10/10
487/487 [==============================] - 48s 99ms/step - loss: 

In [15]:
for src_sample, tgt_sample in train_dataset.take(1): break
model_02(src_sample)
model_02.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [16]:
model_02.compile(loss=loss, optimizer=optimizer)
history_02 = model_02.fit(train_dataset, epochs=10,validation_data=val_dataset,
                    verbose=1)

Epoch 1/10
487/487 [==============================] - 66s 136ms/step - loss: 3.3416 - val_loss: 3.0068
Epoch 2/10
487/487 [==============================] - 65s 134ms/step - loss: 2.9119 - val_loss: 2.8736
Epoch 3/10
487/487 [==============================] - 65s 134ms/step - loss: 2.7792 - val_loss: 2.7947
Epoch 4/10
487/487 [==============================] - 65s 134ms/step - loss: 2.6803 - val_loss: 2.7379
Epoch 5/10
487/487 [==============================] - 65s 134ms/step - loss: 2.5921 - val_loss: 2.6907
Epoch 6/10
487/487 [==============================] - 65s 134ms/step - loss: 2.5088 - val_loss: 2.6477
Epoch 7/10
487/487 [==============================] - 66s 135ms/step - loss: 2.4295 - val_loss: 2.6156
Epoch 8/10
487/487 [==============================] - 65s 134ms/step - loss: 2.3545 - val_loss: 2.5852
Epoch 9/10
487/487 [==============================] - 65s 134ms/step - loss: 2.2823 - val_loss: 2.5562
Epoch 10/10
487/487 [==============================] - 65s 134ms/step - l

In [17]:
for src_sample, tgt_sample in train_dataset.take(1): break
model_12(src_sample)
model_12.summary()

Model: "text_generator_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_4 (LSTM)                multiple                  5246976   
_________________________________________________________________
batch_normalization_1 (Batch multiple                  4096      
_________________________________________________________________
lstm_5 (LSTM)                multiple                  8392704   
_________________________________________________________________
batch_normalization_2 (Batch multiple                  4096      
_________________________________________________________________
dense_3 (Dense)              multiple                  12301025  
Total params: 29,021,153
Trainable params: 29,017,057
Non-trainable params: 4,096
__________________________________

In [18]:
model_12.compile(loss=loss, optimizer=optimizer)
history_12 = model_12.fit(train_dataset, epochs=10,validation_data=val_dataset,
                    verbose=1)

Epoch 1/10
487/487 [==============================] - 68s 141ms/step - loss: 3.1760 - val_loss: 3.0127
Epoch 2/10
487/487 [==============================] - 71s 145ms/step - loss: 2.7179 - val_loss: 2.6953
Epoch 3/10
487/487 [==============================] - 70s 145ms/step - loss: 2.5141 - val_loss: 2.6139
Epoch 4/10
487/487 [==============================] - 70s 144ms/step - loss: 2.3453 - val_loss: 2.5414
Epoch 5/10
487/487 [==============================] - 71s 145ms/step - loss: 2.2012 - val_loss: 2.5045
Epoch 6/10
487/487 [==============================] - 71s 145ms/step - loss: 2.0787 - val_loss: 2.4852
Epoch 7/10
487/487 [==============================] - 71s 146ms/step - loss: 1.9705 - val_loss: 2.4718
Epoch 8/10
487/487 [==============================] - 70s 144ms/step - loss: 1.8732 - val_loss: 2.4611
Epoch 9/10
487/487 [==============================] - 71s 146ms/step - loss: 1.7854 - val_loss: 2.4708
Epoch 10/10
487/487 [==============================] - 70s 144ms/step - l

In [19]:
hidden_size = 2048
h2_model_01 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size, 0, 1)
for src_sample, tgt_sample in train_dataset.take(1): break
h2_model_01(src_sample)
h2_model_01.summary()

Model: "text_generator_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_6 (LSTM)                multiple                  18882560  
_________________________________________________________________
dense_4 (Dense)              multiple                  24590049  
Total params: 46,544,865
Trainable params: 46,544,865
Non-trainable params: 0
_________________________________________________________________


In [20]:
h2_model_01.compile(loss=loss, optimizer=optimizer)
history_h01 = h2_model_01.fit(train_dataset, epochs=10,validation_data=val_dataset,
                    verbose=1)

Epoch 1/10
487/487 [==============================] - 110s 226ms/step - loss: 3.3433 - val_loss: 2.9524
Epoch 2/10
487/487 [==============================] - 112s 230ms/step - loss: 2.8119 - val_loss: 2.7497
Epoch 3/10
487/487 [==============================] - 113s 231ms/step - loss: 2.5762 - val_loss: 2.6141
Epoch 4/10
487/487 [==============================] - 111s 228ms/step - loss: 2.3566 - val_loss: 2.5053
Epoch 5/10
487/487 [==============================] - 111s 227ms/step - loss: 2.1450 - val_loss: 2.4187
Epoch 6/10
487/487 [==============================] - 111s 227ms/step - loss: 1.9438 - val_loss: 2.3504
Epoch 7/10
487/487 [==============================] - 111s 228ms/step - loss: 1.7568 - val_loss: 2.2991
Epoch 8/10
487/487 [==============================] - 112s 229ms/step - loss: 1.5891 - val_loss: 2.2637
Epoch 9/10
487/487 [==============================] - 111s 229ms/step - loss: 1.4438 - val_loss: 2.2456
Epoch 10/10
487/487 [==============================] - 112s 230m

In [21]:
h2_model_02 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size, 0, 2)
for src_sample, tgt_sample in train_dataset.take(1): break
h2_model_02(src_sample)
h2_model_02.summary()

Model: "text_generator_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_7 (LSTM)                multiple                  18882560  
_________________________________________________________________
lstm_8 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense_5 (Dense)              multiple                  24590049  
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


In [22]:
h2_model_02.compile(loss=loss, optimizer=optimizer)
history_h02 = h2_model_02.fit(train_dataset, epochs=10,validation_data=val_dataset,
                    verbose=1)

Epoch 1/10
487/487 [==============================] - 183s 377ms/step - loss: 3.4476 - val_loss: 3.0791
Epoch 2/10
487/487 [==============================] - 184s 378ms/step - loss: 2.9604 - val_loss: 2.8952
Epoch 3/10
487/487 [==============================] - 184s 378ms/step - loss: 2.7786 - val_loss: 2.7885
Epoch 4/10
487/487 [==============================] - 184s 377ms/step - loss: 2.6166 - val_loss: 2.6969
Epoch 5/10
487/487 [==============================] - 184s 379ms/step - loss: 2.4648 - val_loss: 2.6253
Epoch 6/10
487/487 [==============================] - 186s 382ms/step - loss: 2.3198 - val_loss: 2.5603
Epoch 7/10
487/487 [==============================] - 185s 380ms/step - loss: 2.1805 - val_loss: 2.5062
Epoch 8/10
487/487 [==============================] - 185s 380ms/step - loss: 2.0460 - val_loss: 2.4582
Epoch 9/10
487/487 [==============================] - 184s 379ms/step - loss: 1.9153 - val_loss: 2.4193
Epoch 10/10
487/487 [==============================] - 185s 379m

In [23]:
h2_model_11 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size, 1, 1)
for src_sample, tgt_sample in train_dataset.take(1): break
h2_model_11(src_sample)
h2_model_11.summary()

Model: "text_generator_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_9 (LSTM)                multiple                  18882560  
_________________________________________________________________
batch_normalization_3 (Batch multiple                  8192      
_________________________________________________________________
dense_6 (Dense)              multiple                  24590049  
Total params: 46,553,057
Trainable params: 46,548,961
Non-trainable params: 4,096
_________________________________________________________________


In [24]:
h2_model_11.compile(loss=loss, optimizer=optimizer)
history_h11 = h2_model_11.fit(train_dataset, epochs=10,validation_data=val_dataset,
                    verbose=1)

Epoch 1/10
487/487 [==============================] - 112s 230ms/step - loss: 3.4332 - val_loss: 2.9319
Epoch 2/10
487/487 [==============================] - 113s 232ms/step - loss: 2.6655 - val_loss: 2.6380
Epoch 3/10
487/487 [==============================] - 113s 232ms/step - loss: 2.4164 - val_loss: 2.5289
Epoch 4/10
487/487 [==============================] - 113s 231ms/step - loss: 2.2178 - val_loss: 2.4575
Epoch 5/10
487/487 [==============================] - 113s 233ms/step - loss: 2.0488 - val_loss: 2.4281
Epoch 6/10
487/487 [==============================] - 113s 232ms/step - loss: 1.9031 - val_loss: 2.3952
Epoch 7/10
487/487 [==============================] - 114s 233ms/step - loss: 1.7743 - val_loss: 2.3946
Epoch 8/10
487/487 [==============================] - 113s 233ms/step - loss: 1.6574 - val_loss: 2.3834
Epoch 9/10
487/487 [==============================] - 113s 233ms/step - loss: 1.5477 - val_loss: 2.3951
Epoch 10/10
487/487 [==============================] - 113s 232m

In [12]:
hidden_size = 3000
embedding_size = 500

b_model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size, 0, 1)
for src_sample, tgt_sample in train_dataset.take(1): break
b_model(src_sample)
b_model.summary()

Model: "text_generator_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  6000500   
_________________________________________________________________
lstm_6 (LSTM)                multiple                  42012000  
_________________________________________________________________
dense_4 (Dense)              multiple                  36015001  
Total params: 84,027,501
Trainable params: 84,027,501
Non-trainable params: 0
_________________________________________________________________


In [13]:
b_model.compile(loss=loss, optimizer=optimizer)
history_b = b_model.fit(train_dataset, epochs=10,validation_data=val_dataset,
                    verbose=1)

Epoch 1/10
487/487 [==============================] - 174s 357ms/step - loss: 3.2902 - val_loss: 2.8992
Epoch 2/10
487/487 [==============================] - 176s 362ms/step - loss: 2.7179 - val_loss: 2.6425
Epoch 3/10
487/487 [==============================] - 177s 363ms/step - loss: 2.3896 - val_loss: 2.4591
Epoch 4/10
487/487 [==============================] - 177s 363ms/step - loss: 2.0581 - val_loss: 2.3227
Epoch 5/10
487/487 [==============================] - 177s 364ms/step - loss: 1.7477 - val_loss: 2.2271
Epoch 6/10
487/487 [==============================] - 177s 364ms/step - loss: 1.4849 - val_loss: 2.1787
Epoch 7/10
487/487 [==============================] - 177s 364ms/step - loss: 1.2856 - val_loss: 2.1656
Epoch 8/10
487/487 [==============================] - 177s 364ms/step - loss: 1.1532 - val_loss: 2.1786
Epoch 9/10
487/487 [==============================] - 177s 364ms/step - loss: 1.0745 - val_loss: 2.1987
Epoch 10/10
487/487 [==============================] - 177s 364m

### 분석
batch normalization을 사용한 것과 아닌 것을 비교해보면   
사용한 것은 validation loss가 빠르게 수렴하여 overfitting이 되는 순간을 볼 수 있지만   
사용하지 않은 것은 천천히 수렴하여 epoch 10으로는 validation loss가 증가하지 않는다.

그리고 RNN layer size가 1인 경우랑 2인 경우를 비교해보면       
크게 차이가 나지 않는다.오히려 1인 경우보다 2인 경우에 loss만 보면 성능이 안 좋다.

RNN의 상태 크기를 1024, 2048인 경우를 비교해보면
역시 상태 크기가 2048인 경우가 성능이 더욱 좋다.

종합적으로 분석해보면 RNN layer가 2보다는 1인 경우가 좋고   
에폭을 줄이면 batch normalization을 쓰는게 좋고 에폭이 10일 경우 안 쓰는 게 좋다.   
RNN의 상태는 1024보다 2048이 좋다.

마지막 모델은 루브릭 평가를 맞추기 위해서 종합적으로 좋은 하이퍼파라미터에서 조금 더 큰 모델로 만들었다.

### Text generation

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor,tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

generate_text(b_model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [16]:
generate_text(b_model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love is a beautiful thing <end> '

In [17]:
generate_text(b_model, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you re the only one i want <end> '

In [18]:
generate_text(b_model, tokenizer, init_sentence="<start>", max_len=20)

'<start> i m gonna be good to you <end> '

In [19]:
generate_text(b_model, tokenizer, init_sentence="<start> good", max_len=20)

'<start> good morning to chitown where my niggas ride down <end> '

In [20]:
generate_text(b_model, tokenizer, init_sentence="<start> he", max_len=20)

'<start> he s a highway chile <end> '

## 회고 및 루브릭 평가

### 루브릭 평가 항목
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?(텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?)
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?(특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?)
3. 텍스트 생성모델이 안정적으로 학습되었는가?(텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?)

### 평가 항목에 대한 수행
1. 마지막 부분이 text generation에서 6개 문장을 생성하였고 그럴듯한 문장이 생성된 것 같다. 5,6번째 문장은 약간 힙합 느낌인 것 같다.
2. '3. data preprocessing' 부분에서 특수문자 제거, 토크나이저 생성, 패딩 처리를 빠짐 없이 하였다.
3. 마지막 실험에서 텍스트 생성 모델의 validation loss가 6,7,8,9 epoch에서 2.2보다 낮아졌다.

### 회고

노션에 공지된 꼭 포함이 되어야 할 점
- 이번 프로젝트에서 **어려웠던 점,**
- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.
- 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.
- 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**.
- **자기 다짐**

---
- **어려웠던 점**    
루브릭 평가 마지막 부분을 맞추기 위해서 하이퍼 파라미터 탐색하는 것이 힘들었다. 모델을 너무 많이 생성해서 중간에 커널이 메모리를 버티지 못하고 죽었다... 그래서 모델 저장도 안해서 불러오지도 못한다. 짧은 훈련 시간으로 성능을 맞추려고 해서 힘들었는데 그냥 시간을 포기하고 큰 모델로 하이퍼파라미터를 설정하니까 바로 성능이 나와서 조금은 허탈했다.   

---
- **알아낸 점**    
위에서 '분석'에서 하이퍼 파라미터에 따라 모델의 성능(loss)에 대해서 어떤 차이가 있는 지 알 수 있었다. 그리고 NLP 분야애서 batch normalization을 사용하는 지를 알아봤는데 CV 쪽보다는 덜 사용하는 것 같다.   

- **모호한 점**    
NLP에서 사용하는 지표가 따로 있을 거라고 생각하는데 그에 대한 지표를 제시해주지 않아서 정확히 어떻게 평가를 내리는 지 모호하다. 

---
- **시도한 것들**    
위에서 한 여러 모델로 실험을 하였다. 이를 위해서 class를 뜯어 고쳤다. 그걸 기반으로 하이퍼파라미터 탐색을 하였다.   

---
- **우브릭 평가 관련 지표**   
제 예상에는 모두 달성되었다고 생각한다. 그 이유는 위에 있는 **평가 항목에 대한 수행**에 나와있다.  
- **자기 다짐** 및 **나의 생각들**      
생각보다 노드를 하는데 시간이 오래 걸렸다. 먼저, 이전 노드보다 훈련하는데 시간이 오래 걸렸고, 훈련 시간을 오래 걸리지 않게 하기 위해서 최대한 하이퍼파라미터를 높게 하지 않은 것이 문제 였다. 이럴 줄 알았으면 그냥 한 번에 높여서 빨리 끝낼 걸 그랬다. 이번 노드를 진행하다가 메모리를 너무 많이 사용해서 커널이 죽었는데 미리 모델들을 저장하지 않아서 다시 사용할 수 없어서 아쉽다. 이제 노드들을 진행할 때에 훈련 시간이 오래 걸릴 것 같은데 만약을 위해서라도 모델들을 저장해야겠다. 어쨌든 커널이 죽은 덕분에 코드를 실행시키는 순서가 엉망이 되었다. 시간이 아까워서 다시 돌리지 못 했다. 그리고 마크업 문법에서 주피터 노트북에서는 순서 리스트가 0번 가능하지만 github는 그것을 지원하지 않아서 0으로 표기한 것이 1번으로 표기되는 것을 확인했다. 고쳐야지 생각만 하다가 이제야 고치게 되었다. NLP 분야라서 저번에 이야기한 미리 공부한 것이 많이 도움이 되어서 이번에도 쉽게 할 수 있었다. 다만 이제부터는 데이터량이랑 훈련시간이 많이 늘어날 것 같은데 잘 조절할 수 있을지 조금은 걱정이 된다. 